# EXPLORATORY DATA ANALYSIS USING SQL

In [32]:
import pandas as pd
import mysql.connector


db = mysql.connector.connect(host = "",
                            username = "",
                            password = "",  
                            database = "")

cur = db.cursor()

# TOP 5 LOCATIONS OF THE CUSTOMERS

In [33]:
query = """
SELECT customer_city, COUNT(*) AS customer_count
FROM customers
GROUP BY customer_city
ORDER BY customer_count DESC
"""

cur.execute(query)
data = cur.fetchall()

columns = ['Cities', 'Number of Customers']

df = pd.DataFrame(data,columns=columns)
df.head()

,Cities,Number of Customers
0,sao paulo,15540
1,rio de janeiro,6882
2,belo horizonte,2773
3,brasilia,2131
4,curitiba,1521


# TOTAL ORDERS PLACED IN 2017 AND 2018

In [34]:
query="""SELECT 
    YEAR(order_purchase_timestamp) AS order_year,
    COUNT(order_id) AS order_count
    FROM orders
    WHERE YEAR(order_purchase_timestamp) IN (2017, 2018)
    GROUP BY order_year"""

cur.execute(query)
data = cur.fetchall()

columns = ['Year', 'Number of Orders']

df = pd.DataFrame(data,columns=columns)
df.head()

,Year,Number of Orders
0,2017,45101
1,2018,54011


# MOST ORDERED PRODUCT CATEGORY

In [35]:
query="""SELECT 
        p.product_category,o.price,COUNT(*) AS order_count
        FROM 
        order_items o 
        INNER JOIN
        products p
        ON o.product_id=p.product_id
        GROUP BY o.product_id,p.product_category,o.price
        ORDER BY order_count DESC
        LIMIT 1
        """

cur.execute(query)
data = cur.fetchall()
data
columns = ['Category','Price','Number of Orders']

df = pd.DataFrame(data,columns=columns)
df.head()


,Category,Price,Number of Orders
0,Furniture Decoration,69.9,404


# TOTAL SALES PER CATEGORY OF PRODUCTS

In [36]:
query = """ SELECT 
    p.product_category,
    ROUND(SUM(py.payment_value), 2) 
    FROM 
    products p
    JOIN 
    order_items o
    ON p.product_id = o.product_id
    JOIN 
    payments py
    ON py.order_id = o.order_id
    GROUP BY product_category
    """

cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data, columns = ["Category", "Sales"])
df

,Category,Sales
0,perfumery,506738.66
1,Furniture Decoration,1430176.39
2,telephony,486882.05
3,bed table bath,1712553.67
4,automotive,852294.33
...,...,...
69,cds music dvds,1199.43
70,La Cuisine,2913.53
71,Fashion Children's Clothing,785.67
72,PC Gamer,2174.43


# TOTAL ORDERS PER CATEGORY OF PRODUCTS

In [37]:
query = """ SELECT 
    p.product_category,
    COUNT(*)
    FROM 
    products p
    INNER JOIN 
    order_items o
    ON p.product_id = o.product_id
    GROUP BY p.product_category"""

cur.execute(query)
data = cur.fetchall()
data
df = pd.DataFrame(data, columns = ["Category", "Number of Orders"])
df

,Category,Number of Orders
0,HEALTH BEAUTY,9670
1,sport leisure,8641
2,Cool Stuff,3796
3,computer accessories,7827
4,Watches present,5991
...,...,...
69,flowers,33
70,Kitchen portable and food coach,15
71,House Comfort 2,30
72,CITTE AND UPHACK FURNITURE,38


# TOP 3 MODES OF PAYMENT

In [38]:
query = """ SELECT 
    p.payment_type,
    COUNT(*),
    ROUND(SUM(p.payment_value),2)
    FROM 
    payments p 
    GROUP BY p.payment_type"""

cur.execute(query)
data = cur.fetchall()
data
df = pd.DataFrame(data, columns = ["Mode of Payment", "Number of Orders", "Total Sales"])
df.head(3)

,Mode of Payment,Number of Orders,Total Sales
0,credit_card,76795,12542084.19
1,UPI,19784,2869361.27
2,voucher,5775,379436.87


# PERCENTAGE OF MOST PREFFERED NUMBER OF INSTALLMENTS FOR PAYMENT BY CUSTOMERS

In [39]:
query = """ SELECT 
    p.payment_installments,
    COUNT(*) * 100.0 / SUM(COUNT(*)) OVER() AS percentage
    FROM 
    payments p  
    GROUP BY p.payment_installments
    ORDER BY percentage DESC
    """

cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data, columns = ["NUMBER OF INSTALLMENTS","PERCENTAGE"])
df.head(5)

,NUMBER OF INSTALLMENTS,PERCENTAGE
0,1,50.58044
1,2,11.94867
2,3,10.06969
3,4,6.83249
4,10,5.12870


# NUMBER OF ORDERS PER MONTH IN THE YEAR 2017 AND 2018

In [40]:
query="""SELECT 
    YEAR(order_purchase_timestamp) AS order_year,
    MONTH(order_purchase_timestamp) AS order_month,
    COUNT(order_id) AS order_count
    FROM orders
    WHERE YEAR(order_purchase_timestamp) IN (2017, 2018)
    GROUP BY order_year,order_month
    ORDER BY order_year"""

cur.execute(query)
data = cur.fetchall()

columns = ['Year','Month', 'Number of Orders']

df = pd.DataFrame(data,columns=columns)
df

,Year,Month,Number of Orders
0,2017,1,800
1,2017,2,1780
2,2017,3,2682
3,2017,4,2404
4,2017,5,3700
5,2017,6,3245
6,2017,7,4026
7,2017,8,4331
8,2017,9,4285
9,2017,10,4631


# AVERAGE ORDERS PER MONTH IN THE YEAR 2017 AND 2018

In [41]:
query="""SELECT 
         order_year,
         order_month,
         ROUND(AVG(order_count), 2)
         FROM (
            SELECT 
            YEAR(order_purchase_timestamp) AS order_year,
            MONTH(order_purchase_timestamp) AS order_month,
            DATE(order_purchase_timestamp) AS order_date,
            COUNT(order_id) AS order_count
            FROM orders
            WHERE YEAR(order_purchase_timestamp) IN (2017, 2018)
            GROUP BY order_year, order_month,order_date
            ) AS order_counts
        GROUP BY order_year, order_month
        ORDER BY order_year, order_month
      """

cur.execute(query)
data = cur.fetchall()

columns = ['Year','Month','Average Number of Orders']

df = pd.DataFrame(data,columns=columns)
df

,Year,Month,Average Number of Orders
0,2017,1,29.63
1,2017,2,63.57
2,2017,3,86.52
3,2017,4,80.13
4,2017,5,119.35
5,2017,6,108.17
6,2017,7,129.87
7,2017,8,139.71
8,2017,9,142.83
9,2017,10,149.39


# TOP 3 CATEGERIES WITH HIGHER SALES


In [42]:
query = """SELECT UPPER(p.product_category),
            ROUND( ( SUM(py.payment_value)/ (SELECT SUM( payment_value) FROM payments) )* 100 ,2) AS sales_percentage
            FROM products p
            INNER JOIN order_items o
            ON o.product_id=p.product_id
            INNER JOIN payments py
            ON o.order_id=py.order_id
            GROUP BY p.product_category
            ORDER BY sales_percentage DESC         
        """

cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data,columns = ["Category", "Sales Percentage"])
df.head()

,Category,Sales Percentage
0,BED TABLE BATH,10.70
1,HEALTH BEAUTY,10.35
2,COMPUTER ACCESSORIES,9.90
3,FURNITURE DECORATION,8.93
4,WATCHES PRESENT,8.93


# TOTAL REVENUE GENERATED BY EACH SELLER

In [43]:
query = """SELECT
            o.seller_id,
            DENSE_RANK() OVER(ORDER BY ROUND(SUM(p.payment_value),2) DESC) denserank,
            ROUND(SUM(p.payment_value),2) revenue
            FROM order_items o            
            INNER JOIN
            payments p
            ON p.order_id=o.order_id
            GROUP BY o.seller_id
        """
cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data,columns = ["Seller ID","Rank","Total Revenue by Seller"])
df.head()

,Seller ID,Rank,Total Revenue by Seller
0,7c67e1448b00f6e969d365cea6b010ab,1,507166.91
1,1025f0e2d44d7041d6cf58b6550e0bfa,2,308222.04
2,4a3ca9315b744ce9f8e9374361493884,3,301245.27
3,1f50f920176fa81dab994f9023523100,4,290253.42
4,53243585a1d6dc2643021fd1853d8905,5,284903.08


# MOVING AVERAGE OF SALES FOR EACH CUSTOMER OVER THEIR ORDER HISTORY

In [44]:
query = """
        SELECT id,opt,val,AVG(val) OVER (PARTITION BY id ORDER BY opt) as mov_avg
        FROM
        (SELECT o.customer_id as id,o.order_purchase_timestamp as opt,p.payment_value as val
        FROM payments p
        INNER JOIN
        orders o
        ON p.order_id=o.order_id) 
        as reqdata
        """
cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data,columns = ["Customer ID","Order Purchase Timestamp","Payment Value","Moving Average"])
df.head()

,Customer ID,Order Purchase Timestamp,Payment Value,Moving Average
0,00012a2ce6f8dcda20d059ce98491703,2017-11-14 16:08:26,114.74,114.739998
1,000161a058600d5901f007fab4c27140,2017-07-16 09:40:32,67.41,67.410004
2,0001fd6190edaaf884bcaf3d49edf079,2017-02-28 11:06:43,195.42,195.419998
3,0002414f95344307404f0ace7a26f1d5,2017-08-16 13:09:20,179.35,179.350006
4,000379cdec625522490c315e70c7a9fb,2018-04-02 13:42:17,107.01,107.010002


# CUMULATIVE SALES PER MONTH YEAR WISE 

In [45]:
query="""SELECT
    year(o.order_purchase_timestamp) AS year,
    month(o.order_purchase_timestamp) AS month,
    ROUND(SUM(p.payment_value), 2) AS sales_per_month,
    SUM(ROUND(SUM(p.payment_value), 2)) OVER (PARTITION BY year(o.order_purchase_timestamp) ORDER BY year(o.order_purchase_timestamp), month(o.order_purchase_timestamp)) AS cumulative_sales
    FROM payments p
    INNER JOIN orders o ON o.order_id = p.order_id
    GROUP BY year(o.order_purchase_timestamp), month(o.order_purchase_timestamp)
    ORDER BY year, month; 
    """
cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data,columns = ["Year","Month","Sales","Cumulative Sales"])
df


,Year,Month,Sales,Cumulative Sales
0,2016,9,252.24,252.24
1,2016,10,59090.48,59342.72
2,2016,12,19.62,59362.34
3,2017,1,138488.04,138488.04
4,2017,2,291908.01,430396.05
5,2017,3,449863.60,880259.65
6,2017,4,417788.03,1298047.68
7,2017,5,592918.82,1890966.50
8,2017,6,511276.38,2402242.88
9,2017,7,592382.92,2994625.80


# CUSTOMERS LOCATION VS MOST PURCHASED PRODUCT CATEGORY OF THAT LOCATIONS

In [46]:
query=""" WITH total_purchase_category_wise AS
    (SELECT 
    c.customer_state,
    p.product_category,
    COUNT(*) as count_purchase,
    RANK() OVER(PARTITION BY c.customer_state ORDER BY COUNT(*) DESC) as rn
    FROM 
    customers c
    INNER JOIN orders o ON o.customer_id = c.customer_id
    INNER JOIN order_items ot ON ot.order_id = o.order_id
    INNER JOIN products p ON p.product_id = ot.product_id
    GROUP BY c.customer_state,p.product_category)
    
    SELECT
    customer_state,
    UPPER(product_category),
    count_purchase
    FROM total_purchase_category_wise
    WHERE rn=1 
       """
cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data,columns = ["Customers Location","Category of Purchase","Total purchases"])
df.head(20)

,Customers Location,Category of Purchase,Total purchases
0,AC,FURNITURE DECORATION,12
1,AL,HEALTH BEAUTY,63
2,AM,HEALTH BEAUTY,20
3,AP,HEALTH BEAUTY,10
4,BA,HEALTH BEAUTY,350
5,CE,HEALTH BEAUTY,167
6,DF,HEALTH BEAUTY,246
7,ES,BED TABLE BATH,225
8,GO,BED TABLE BATH,235
9,MA,HEALTH BEAUTY,89
